# NetCDF and CF: The Basics



### Abstract and other notes 
This workshop will teach some of the basics of CF metadata for netCDF data files
with some hands-on work available in Jupyter Notebooks using Python. Along with
introduction to netCDF and CF, we will introduce the CF data model and discuss
some netCDF implementation details to consider when deciding how to write data
with CF and netCDF. We will cover gridded data as well as in situ data
(stations, soundings, etc.) and touch on storing geometries data in CF.

Assume: Basic understanding of netCDF and CF (what they are and how they work
together?)

Target Audience: Data producer or manager
- Have data they want to (or have been told they should) write to CF compliant
  netCDF files

### Some References

- See CF Conventions doc ([1.7](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html))
- See Jonathan Gregory's old [CF presentation](http://cfconventions.org/Data/cf-documents/overview/viewgraphs.pdf)
- See [CF presentation](https://docs.google.com/presentation/d/1OImxWBNxyj-zdreIarH5GSIuDyREGB62rDah19g6M94/edit#) I gave at Oct 2018 nc training workshop
- See  NASA ESDS “Dataset Interoperability Recommendations for Earth Science” ([web page](https://earthdata.nasa.gov/user-resources/standards-and-references/dataset-interoperability-recommendations-for-earth-science))
- See CF Data Model (cfdm) python package [tutorial](https://ncas-cms.github.io/cfdm/tutorial.html)
- See Tim Whiteaker's cfgeom python package (GitHub [repo](https://github.com/twhiteaker/CFGeom))([tutorial]( https://twhiteaker.github.io/CFGeom/tutorial.html))

## Overview of netCDF and CF



### netCDF Data Model

### CF Data Model


## netCDF coordinate variables

Coordinate variables allow 

### Latitude, Longitude, and Vertical

For latitude, include units attribute with a 'degrees_north' value.
```
float lat(lat) ;
  lat:long_name = "latitude" ;
  lat:units = "degrees_north" ;
  lat:standard_name = "latitude" ;
```

For longitude, a units attribute must be included with a 'degrees_east' value. A
standard name of 'longitude' is another mechanism for recognizing longitude.

```
float lon(lon) ;
  lon:long_name = "longitude" ;
  lon:units = "degrees_east" ;
  lon:standard_name = "longitude" ;
```
A vertical coordinate can be recognized by its units and the 'positive'
attribute with value of 'up' or 'down'.

Height, depth
```
axis_name:units = "meters" ;
axis_name:positive = "down" ;
```
Pressure
```
float pres(pres) ;
  pres:long_name = "pressure" ;
  pres:units = "hPa" ;
```

### Time

Time coordinates must include a 'units' attribute with a string value with a
form similar to
    'seconds since 2019-01-06 12:00:00.00'

'seconds', 'minutes', 'hours', and 'days' are the most commonly used units for
time. Due to the variable length of months and years, they are not recommended.

### Direct axis identification
The 'axis' attribute can be used with a value of 'X', 'Y', 'Z', and 'T' to
simplify identification of space and time coordinates and to identify generic
spatial coordinates, e.g., a projected coordinate system given by a 'grid_mapping'.


## Units

The 'units' attribute should be used for all variables that represent a
dimensional quantity. With only a few exceptions, the value of the `units`
attribute must be recognizable by the Unidata Udunits package.



##- Gridded data
- lat/lon 1-D coordinate variables
- x/y 1-D coordinate variables (with 2-D lat/lon)
  - "coordinates" attribute
  - Grid Mapping
- Cell bounds (and cell measures?)
  - an example
- Cell methods
  - an example
- ??"climatology" attribute??s


## Observational data (CF DSG)
- Overview of Point, station, sounding, trajectory
- Examples
- Reference to [NOAA NCEI netCDF Templates](https://www.nodc.noaa.gov/data/formats/netcdf/v2.0/)


## Geometries

- Overview
- Example
- Link to cfgeom python package


## CF Standard Names
- Overview:
  - Controlled vocabulary of names that describe physical quantity
    - (name, description, canonical units)
  - Canonical units (units used must be convertable to)
  - Help users to decide if data from two sources are comparable
    

## Other topics
- Missing data
  - _FillValue, missing_data, valid_*
- Packed data (scale_factor/add_offset)
- Chunking and compression
- CRS, shape of earth, horizontal and vertial datum


## Recommendations from NASA 
- [NASA Dataset Interoperability Recommendations for Earth Science](https://earthdata.nasa.gov/esdis/eso/standards-and-references/dataset-interoperability-recommendations-for-earth-science) - two PDF documents.
- NASA Data Prodcut Developer's Guide (from the [previous session](https://2019esipsummermeeting.sched.com/event/PtOg/data-product-developers-guide-workshop))
